<a href="https://colab.research.google.com/github/ThaysonScript/projeto_rag_llama/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregando chaves de api

In [3]:
!pip install langchain-groq
!pip install langchain_community
!pip install langchain_text_splitter
!pip install langchain_huggingface
!pip install langgraph
!pip install python-dotenv
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17
ERROR: Could not find a versi

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from dotenv import load_dotenv
import os

os.environ['GROQ_API_KEY'] = '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'

# Definindo e chamando modelos que vão ser usados

In [6]:
from langchain_groq import ChatGroq

class Modelos:
    def __init__(self):
        self.models = {
            'google_gemma': 'gemma2-9b-it',
            'meta_llama': 'llama-3.3-70b-versatile',
            'deep_seek': 'deepseek-r1-distill-qwen-32b'
        }

        self.google_llm = ChatGroq(model=self.models['google_gemma'])
        self.meta_llm = ChatGroq(model=self.models['meta_llama'])
        self.deep_seek_llm = ChatGroq(model=self.models['deep_seek'])

    def imprimir_llms_carregadas(self):
        print(f'GOOGLE_LLM: {self.google_llm}\n')
        print(f'META_LLM: {self.meta_llm}\n')
        print(f'DEEP_SEEK_LLM: {self.deep_seek_llm}\n')

new_obj = Modelos()
new_obj.imprimir_llms_carregadas()

GOOGLE_LLM: client=<groq.resources.chat.completions.Completions object at 0x7c338fa91fd0> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c338faa3210> model_name='gemma2-9b-it' model_kwargs={} groq_api_key=SecretStr('**********')

META_LLM: client=<groq.resources.chat.completions.Completions object at 0x7c33a49ed690> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c33a44ae850> model_name='llama-3.3-70b-versatile' model_kwargs={} groq_api_key=SecretStr('**********')

DEEP_SEEK_LLM: client=<groq.resources.chat.completions.Completions object at 0x7c338fabc190> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c338fae6fd0> model_name='deepseek-r1-distill-qwen-32b' model_kwargs={} groq_api_key=SecretStr('**********')



# Carregar Lista de PDFS dos Prospectos

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
import os

# Lista para armazenar os documentos carregados
diretorio_base='/content/drive/MyDrive/topicos_ia/data/minuta'
docs_list = []
i = 1

# Itera sobre os arquivos no diretório e carrega os PDFs
for nome_arquivo in os.listdir(diretorio_base):
    if nome_arquivo.endswith(".pdf"):  # Verifica se o arquivo é um PDF
        caminho_pdf = os.path.join(diretorio_base, nome_arquivo)

        loader = PyPDFLoader(caminho_pdf)

        print(f'loader: {loader}\n\n')

        docs = await loader.aload()

        print(f'docs_loader: {docs[0].page_content[:100]}\n\n')
        print(docs[0].metadata)

        docs_list.append(docs)

        print(i)
        i += 1

    if i == 15:
      break


loader: <langchain_community.document_loaders.pdf.PyPDFLoader object at 0x7c338fd14d10>




# Dividir Texto em partes menores

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

spliters = list()

for docs in docs_list:
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=150,
    add_start_index=True
  ).split_documents(docs)

  spliters.extend(text_splitter)

print(f'Quantidade de partes: {len(spliters)}')

# Instanciando Embedding

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

print(embeddings)

# Usando InMemoryVectorStore para armazenar os embeddings

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Criando o vector store na memória (evita FAISS caso não queira instalar)
vector_store = InMemoryVectorStore(embeddings)

# Configuração do tamanho do lote e número de threads
batch_size = 1000  # Processa poucos documentos por vez para reduzir o uso de RAM
num_threads = 5  # Número de lotes processados simultaneamente
document_ids = []

# Função para processar um lote e armazenar no vector store
def process_batch(start_idx):
    batch = spliters[start_idx : start_idx + batch_size]
    return vector_store.add_documents(documents=batch)

# Cria um pool de threads para paralelizar a execução
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = {executor.submit(process_batch, i): i for i in range(0, len(spliters), batch_size)}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processando lotes em paralelo"):
        document_ids.append(future.result())  # Armazena os IDs processados

print("✅ Processamento concluído com paralelismo!")

# Prompt

In [ ]:
from langchain import hub

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

In [ ]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = new_obj.deep_seek_llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
result = graph.invoke({"question": "me de informações gerais e resumidas do documento"})

# print(f'Context: {result["context"]}\n\n')
# print(f'Answer: {result["answer"]}')
print(result)
print('\n\n')
print(result["answer"])

# STREAMLIT

In [ ]:
!pip install streamlit pyngrok

In [ ]:
%%writefile app.py
import streamlit as st

st.title("Minha App Streamlit no Colab")
st.write("Olá, mundo! Essa aplicação está rodando no Colab via ngrok.")
